In [13]:
import joblib
import re
import numpy as np
import pandas as pd

In [14]:
drug_profiles = joblib.load(
    r"C:\Users\likit\Desktop\drug_decision_support\models\drug_profiles.pkl"
)
severity_model = joblib.load(
    r"C:\Users\likit\Desktop\drug_decision_support\models\severity_model.pkl"
)
vectorizer = joblib.load(
    r"C:\Users\likit\Desktop\drug_decision_support\models\tfidf_vectorizer.pkl"
)

print("Loaded drugs:", len(drug_profiles))
print("Sample drugs:", list(drug_profiles.keys())[:10])



Loaded drugs: 99
Sample drugs: ['ortho-tri-cyclen', 'prilosec', 'lyrica', 'propecia', 'vyvanse', 'elavil', 'xanax', 'claritin', 'effexor-xr', 'neurontin']


In [15]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)
    return text


In [16]:
def extract_keywords(text):
    return set(clean_text(text).split())



In [17]:
def match_expectation(drug, symptoms):
    drug = normalize_drug_name(drug)

    if drug not in drug_profiles:
        return "No Historical Data for Drug", []

    # Expand phrase-level effects to tokens
    expected_effects = set()
    for effect in drug_profiles[drug]["common_effects"]:
        expected_effects.update(effect.split())

    symptom_terms = extract_keywords(symptoms)
    matched = symptom_terms.intersection(expected_effects)

    if len(matched) == 0:
        return "Unexpected", []
    elif len(matched) < len(symptom_terms) / 2:
        return "Partially Expected", list(matched)
    else:
        return "Expected", list(matched)


In [18]:
severity_map = {
    0: "No Side Effects",
    1: "Mild",
    2: "Moderate",
    3: "Severe",
    4: "Extremely Severe"
}


In [19]:
def predict_severity(symptoms):
    cleaned = clean_text(symptoms)
    X = vectorizer.transform([cleaned])

    pred_class = severity_model.predict(X)[0]
    prob = severity_model.predict_proba(X)[0][pred_class]

    return severity_map[pred_class], round(prob * 100, 2), pred_class


In [20]:
def decide_action(expectation, severity_class, confidence):
    """
    Returns action guidance
    """
    if expectation == "Unexpected" and severity_class >= 2:
        return "Seek Medical Attention Immediately"

    if severity_class >= 4:
        return "Seek Medical Attention Immediately"

    if severity_class == 3:
        return "Seek Medical Attention"

    if severity_class == 2:
        if confidence >= 60:
            return "Monitor Closely"
        else:
            return "Reassess Symptoms"

    if severity_class <= 1:
        return "Safe"

    return "Monitor"


In [21]:
drug_aliases = {
    "ibuprofen": ["ibuprofen", "advil", "motrin"],
    "paracetamol": ["paracetamol", "acetaminophen", "tylenol"]
}

def normalize_drug_name(drug):
    drug = drug.lower().strip()
    for canonical, aliases in drug_aliases.items():
        if drug in aliases:
            return canonical
    return drug


In [22]:
def drug_decision_support(drug, symptoms):
    # Normalize drug name first
    drug_norm = normalize_drug_name(drug)

    # Expectation matching
    expectation, matched_terms = match_expectation(drug_norm, symptoms)

    # Severity prediction
    severity_text, confidence, severity_class = predict_severity(symptoms)

    # Action decision
    action = decide_action(expectation, severity_class, confidence)

    return {
        "Drug (input)": drug,
        "Drug (normalized)": drug_norm,
        "Reported Symptoms": symptoms,
        "Expected Side Effects Match": expectation,
        "Matched Expected Terms": matched_terms,
        "Predicted Severity": severity_text,
        "Confidence (%)": confidence,
        "Recommended Action": action
    }


In [23]:
test_cases = [
    ("lyrica", "dizziness and nausea"),
    ("lyrica", "severe chest pain and fainting"),
]

for d, s in test_cases:
    print(drug_decision_support(d, s))
    print("-" * 60)



{'Drug (input)': 'lyrica', 'Drug (normalized)': 'lyrica', 'Reported Symptoms': 'dizziness and nausea', 'Expected Side Effects Match': 'Unexpected', 'Matched Expected Terms': [], 'Predicted Severity': 'Moderate', 'Confidence (%)': np.float64(31.74), 'Recommended Action': 'Seek Medical Attention Immediately'}
------------------------------------------------------------
{'Drug (input)': 'lyrica', 'Drug (normalized)': 'lyrica', 'Reported Symptoms': 'severe chest pain and fainting', 'Expected Side Effects Match': 'Unexpected', 'Matched Expected Terms': [], 'Predicted Severity': 'Extremely Severe', 'Confidence (%)': np.float64(58.86), 'Recommended Action': 'Seek Medical Attention Immediately'}
------------------------------------------------------------
